In [1]:
import os
from os.path import dirname
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from utils.dataloader.TSC_data_loader import TSC_data_loader
from Classifiers.OS_CNN.OS_CNN_Structure_build import generate_layer_parameter_list
from Classifiers.OS_CNN.log_manager import eval_condition, eval_model, save_to_log
from Classifiers.OS_CNN.OS_CNN import OS_CNN


# model saved in result log folder
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'
dataset_path = dirname("./Example_Datasets/UCRArchive_2018/")
dataset_name = 'FiftyWords'

#batch_size for test
batch_size = 16
print('test dataset:', dataset_name)

test dataset: FiftyWords


In [2]:
# load data
X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('code is running on: ',device)    
    
# put data to GPU
X_train = torch.from_numpy(X_train)
X_train.requires_grad = False
X_train = X_train.to(device)
X_train = X_train.unsqueeze_(1)
y_train = torch.from_numpy(y_train).to(device)

X_test = torch.from_numpy(X_test)
X_test.requires_grad = False
X_test = X_test.to(device)
X_test = X_test.unsqueeze_(1)
y_test = torch.from_numpy(y_test).to(device)
input_shape = X_train.shape[-1]
n_class = max(y_train) + 1
print('the size of inpute dataset is:',input_shape)
print('number of class is:',n_class)
    
    
#build dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=False)

code is running on:  cuda:0
the size of inpute dataset is: 270
number of class is: tensor(50, device='cuda:0')


In [3]:
#net parameter
paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128] 
Max_kernel_size = 89

# calcualte network structure
receptive_field_shape= min(int(X_train.shape[-1]/4),Max_kernel_size)
print('the shape of inpute data is:',X_train.shape)
print('the max size of kernel is:', receptive_field_shape)
layer_parameter_list = generate_layer_parameter_list(1,receptive_field_shape,paramenter_number_of_layer_list)
print('the Network structure for',dataset_name,'is:')
print(layer_parameter_list)


the shape of inpute data is: torch.Size([450, 1, 270])
the max size of kernel is: 67
the Network structure for FiftyWords is:
[[(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 5), (1, 1, 7), (1, 1, 11), (1, 1, 13), (1, 1, 17), (1, 1, 19), (1, 1, 23), (1, 1, 29), (1, 1, 31), (1, 1, 37), (1, 1, 41), (1, 1, 43), (1, 1, 47), (1, 1, 53), (1, 1, 59), (1, 1, 61), (1, 1, 67)], [(20, 20, 1), (20, 20, 2), (20, 20, 3), (20, 20, 5), (20, 20, 7), (20, 20, 11), (20, 20, 13), (20, 20, 17), (20, 20, 19), (20, 20, 23), (20, 20, 29), (20, 20, 31), (20, 20, 37), (20, 20, 41), (20, 20, 43), (20, 20, 47), (20, 20, 53), (20, 20, 59), (20, 20, 61), (20, 20, 67)], [(400, 20, 1), (400, 20, 2)]]


In [4]:
# find model path
best_model_path = Result_log_folder +dataset_name+'/'+dataset_name+'Best_model'

# build os-cnn net and load weight
torch_FCN = OS_CNN(layer_parameter_list, n_class.item(), False).to(device)
torch_FCN.load_state_dict(torch.load(best_model_path, map_location=device))
torch_FCN.eval()

acc_test = eval_model(torch_FCN, test_loader)
 
sentence = str(acc_test)
torch.cuda.empty_cache()
print(dataset_name,'test acc = ', acc_test)

FiftyWords test acc =  0.8153846153846154
